<a href="https://colab.research.google.com/github/Iffraah96/Deep-Learning-AI-ITAI-2376-/blob/main/Virtual_AI_Research_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#MODULE 1: Input Processor
#Classify user input into one (or more) of the following intents:

#1. "search": Find papers based on a topic.
#2. "summarize": Summarize a given paper or abstract.
#3. "connect": Find relationships between sources.
#4. "cite": Generate APA/MLA citations.

import re

def input_processor(user_input):
    user_input = user_input.lower().strip()

    # Define simple patterns for classification
    tasks = {
        "search": [
            r'\b(find|search|get|look for|collect)\b.*\b(paper|article|source|research)\b',
            r'\btopic\b.*\b(is|about)\b',
        ],
        "summarize": [
            r'\b(summarize|summarise|give me a summary|what is this about)\b',
            r'\bsummary of\b'
        ],
        "connect": [
            r'\b(connection|relationship|link|relate)\b.*\b(source|paper|idea)\b',
            r'\bhow are\b.*\b(related|connected)\b'
        ],
        "cite": [
            r'\b(cite|citation|apa|mla|format)\b',
            r'\bhow do i cite\b'
        ]
    }

    detected = []
    for task, patterns in tasks.items():
        for pattern in patterns:
            if re.search(pattern, user_input):
                detected.append(task)
                break  # no need to match more patterns for same task

    if not detected:
        detected = ["search"]  # Default action: search if unsure

    return detected

#Example Use Cases
print(input_processor("Can you help me find research on AI in education?"))

print(input_processor("Summarize this article on bias in facial recognition."))

print(input_processor("What's the connection between these two papers?"))

print(input_processor("Format this citation in APA style"))


['summarize']
['search']
['cite']
